# **Predicción del precio de autos usado**

## **Introducción**
* Objetivo: predecir el precio de autos usados en Alemania.
* ¿Por qué es útil? Ayuda a compradores y vendedores a identificar precios justos.
* Enfoque: modelos de regresión (Regresión Lineal, Árboles de Decisión, Bosque Aleatorio).

## **Exploración y limpieza de datos**
* Cargar el conjunto de datos: ```cars.csv```.
* Revisar tamaño, columnas y tipos de datos.
* Corregir problemas:
    * Valores faltantes.
    * Duplicados.
    * Valores fuera de rango (por ejemplo, price = 0, year = 1000).
    * Columnas irrelevantes o mal traducidas (algunas pueden estar en alemán).

* Traducir nombres de columnas si es necesario.

In [1]:
# Cargar librerías
import pandas as pd 
import numpy as np 
import re

In [2]:
# Ruta del archivo de los datos de los autos
path_file = "../data/raw/cars.csv"

# Cargar el conjunto de datos
df = pd.read_csv(path_file)
display(df.head())

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,0,alfa-romeo,Alfa Romeo GTV,red,10/1995,1995,1300,148,201,Manual,Petrol,"10,9 l/100 km",260 g/km,160500.0,2.0 V6 TB
1,1,alfa-romeo,Alfa Romeo 164,black,02/1995,1995,24900,191,260,Manual,Petrol,NaN,- (g/km),190000.0,"Q4 Allrad, 3.2L GTA"
2,2,alfa-romeo,Alfa Romeo Spider,black,02/1995,1995,5900,110,150,Unknown,Petrol,NaN,- (g/km),129000.0,ALFA ROME 916
3,3,alfa-romeo,Alfa Romeo Spider,black,07/1995,1995,4900,110,150,Manual,Petrol,"9,5 l/100 km",225 g/km,189500.0,2.0 16V Twin Spark L
4,4,alfa-romeo,Alfa Romeo 164,red,11/1996,1996,17950,132,179,Manual,Petrol,"7,2 l/100 km",- (g/km),96127.0,"3.0i Super V6, absoluter Topzustand !"


In [3]:
# Eliminar la columna de índice redundante
df = df.drop(columns=["Unnamed: 0"])

# Información general del DataFrame
print("Información general del DataFrame")
print(df.info())

Información general del DataFrame
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251079 entries, 0 to 251078
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   brand                     251079 non-null  object 
 1   model                     251079 non-null  object 
 2   color                     250913 non-null  object 
 3   registration_date         251075 non-null  object 
 4   year                      251079 non-null  object 
 5   price_in_euro             251079 non-null  object 
 6   power_kw                  250945 non-null  object 
 7   power_ps                  250950 non-null  object 
 8   transmission_type         251079 non-null  object 
 9   fuel_type                 251079 non-null  object 
 10  fuel_consumption_l_100km  224206 non-null  object 
 11  fuel_consumption_g_km     251079 non-null  object 
 12  mileage_in_km             250927 non-null  float64
 13  offer_desc

In [4]:
# Valores nulos por columna
print("Valores nulos por columna:")
print(df.isnull().sum())

# Número de filas duplicadas
print("\nNúmero de filas duplicadas:")
print(df.duplicated().sum())

Valores nulos por columna:
brand                           0
model                           0
color                         166
registration_date               4
year                            0
price_in_euro                   0
power_kw                      134
power_ps                      129
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km    26873
fuel_consumption_g_km           0
mileage_in_km                 152
offer_description               1
dtype: int64

Número de filas duplicadas:
6353


In [5]:
# Identificar columnas categóricas y numéricas
categorical_columns = df.select_dtypes(include="object").columns
numerical_columns = df.select_dtypes(include="number").columns

print("Columnas categóricas:")
print(categorical_columns)

print("Columnas numéricas:")
print(numerical_columns)

# Mostrar número de valores únicos por columna categórica
print("\nValores únicos por columna categórica:")
print(df[categorical_columns].nunique())

Columnas categóricas:
Index(['brand', 'model', 'color', 'registration_date', 'year', 'price_in_euro',
       'power_kw', 'power_ps', 'transmission_type', 'fuel_type',
       'fuel_consumption_l_100km', 'fuel_consumption_g_km',
       'offer_description'],
      dtype='object')
Columnas numéricas:
Index(['mileage_in_km'], dtype='object')

Valores únicos por columna categórica:
brand                           47
model                         1312
color                           14
registration_date              433
year                            91
price_in_euro                18228
power_kw                       596
power_ps                       578
transmission_type                4
fuel_type                      136
fuel_consumption_l_100km       621
fuel_consumption_g_km         1500
offer_description           200945
dtype: int64


In [6]:
# Inspeccionar valores únicos de "registration_date"
df["registration_date"].unique()

array(['10/1995', '02/1995', '07/1995', '11/1996', '04/1996', '12/1996',
       '07/1996', '01/1996', '02/1996', '10/1996', '05/1997', '06/1997',
       '10/1997', '03/1997', '01/1997', '02/1997', '04/1997', '07/1997',
       '09/1997', '07/1998', '05/1998', '01/1998', '11/1998', '09/1998',
       '07/1999', '02/1999', '06/1999', '12/1999', '05/1999', '01/1999',
       '04/1999', '03/1999', '10/2000', '06/2000', '05/2000', '01/2000',
       '04/2000', '11/2000', '03/2000', '02/2000', '09/2001', '02/2001',
       '03/2001', '01/2001', '11/2001', '12/2001', '04/2001', '05/2001',
       '07/2001', '06/2001', '10/2002', '12/2002', '01/2002', '09/2002',
       '11/2002', '06/2002', '03/2002', '02/2002', '05/2003', '10/2003',
       '07/2003', '02/2003', '01/2003', '03/2003', '08/2003', '06/2003',
       '03/2004', '02/2004', '09/2004', '04/2004', '08/2004', '01/2004',
       '11/2004', '06/2004', '10/2004', '12/2004', '07/2004', '05/2004',
       '10/2005', '06/2005', '07/2005', '09/2005', 

In [7]:
# Inspeccionar "mileage_in_km"
df["mileage_in_km"].value_counts()

mileage_in_km
10.0       6982
50.0       1533
15.0       1216
20.0       1208
5.0        1107
           ... 
20608.0       1
14296.0       1
34093.0       1
20435.0       1
17166.0       1
Name: count, Length: 71766, dtype: int64

In [8]:
# Extraer el año de "registration_date"
pattern = r"^(0[1-9]|1[0-2])\/(\d{4})$"

def extract_year(date):
    """
    Extraer el año del formato MM/YYYY. Devuelve NaN si el patrón no coincide.
    """
    date_str = str(date)
    match = re.match(pattern, date_str)
    if match:
        return match.group(2)
    else:
        return np.nan
    
df["registration_date"] = df["registration_date"].apply(extract_year)

# Convertir a numérico
df["registration_date"] = pd.to_numeric(df["registration_date"], errors="coerce")

# Covertir el año numérico y filtrar valores realistas
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["year"] = df["year"].where((df["year"] >= 1995) & (df["year"] <= 2023))

# Convertir columnas numéricas
df["price_in_euro"] = pd.to_numeric(df["price_in_euro"], errors="coerce")
df["power_kw"] = pd.to_numeric(df["power_kw"], errors="coerce")
df["power_ps"] = pd.to_numeric(df["power_ps"], errors="coerce")

# Limpiar "fuel_consumption_l_100km"
df["fuel_consumption_l_100km"] = df["fuel_consumption_l_100km"].str.extract(r"^(\d+,\d)\s*l/100\s*km$")[0]
df["fuel_consumption_l_100km"] = df["fuel_consumption_l_100km"].str.replace(",", ".", regex=False).astype(float)

# Limpiar "fuel_consumption_g_km"
df["fuel_consumption_g_km"] = df["fuel_consumption_g_km"].str.extract(r"^(\d+)\s*g/km$")[0]
df["fuel_consumption_g_km"] = df["fuel_consumption_g_km"].str.replace(",", ".", regex=False).astype(float)

# Inspeccionar columnas categóricas
print("Tipos de transisión únicos:")
print(df["transmission_type"].unique())

print("\nTipos de combustible únicos:")
print(df["fuel_type"].unique())


Tipos de transisión únicos:
['Manual' 'Unknown' 'Automatic' 'Semi-automatic']

Tipos de combustible únicos:
['Petrol' 'Diesel' 'Hybrid' 'LPG' 'Other' '07/2004' '74.194 km'
 '110.250 km' '06/2014' 'CNG' 'Diesel Hybrid' '12/2016' 'Automatic'
 'Electric' '12/2019' 'Unknown' '06/2023' 'Ethanol' 'Manual' '300.000 km'
 '264.000 km' 'KETTE NEUE' '108.313 km' '05/2009' '180.000 km' '04/2013'
 '03/2014' '08/2014' '01/2016' '03/2017' '04/2008' '07/2007' '145.500 km'
 '12/2012' '25890' '10/2022' '06/2004' '09/2009' '12/2014' '02/2017'
 '12890' '11/2018' '08/2018' '03/2019' '19450' '11/2021' '20.600 km'
 'Hydrogen' '07/2022' '05/2015' '03/2018' '04/2022' '160.629 km'
 '144.919 km' '02/1996' '04/2000' '200.000 km' '06/2009' '185.500 km'
 '13000' '05/2012' '11/2014' '10/2015' '350.000 km' '49.817 km' '34900'
 '35.487 km' '03/2021' '26890' '26990' '4.000 km' '11/2005' '07/2005'
 '08/2011' '02/2011' '03/2011' '10/2013' '09/2015' '02/2018' '02/2023'
 '08/2017' '99.900 km' '67780' '30.450 km' '62.880 km

In [9]:
# Definir tipos de combustible válidos
valid_fuel_types = [
    "Petrol", "Diesel", "Hybrid", "Diesel Hybrid",
    "Electric", "LPG", "CNG", "Hydrogen", "Ethanol", "Other"
]

# Reemplazar valores inválidos con "Unknown"
df["fuel_type"] = df["fuel_type"].apply(lambda x: x if x in valid_fuel_types else "Unknown")

print("Distribución de tipos de combustible después de la limpieza")
print(df["fuel_type"].value_counts())

Distribución de tipos de combustible después de la limpieza
fuel_type
Petrol           143280
Diesel            86421
Hybrid            12607
Electric           5967
LPG                1255
CNG                 508
Diesel Hybrid       476
Unknown             295
Other               178
Hydrogen             82
Ethanol              10
Name: count, dtype: int64


In [10]:
# Evitar trabajar con un slice
df = df.copy()

# Conversión de kW a PS
df["calculated_power_ps"] = round(df["power_kw"] * 1.359621, 0)
df["difference_power_ps_and_calculated_power_ps"] = df["calculated_power_ps"] - df["power_ps"]
df = df.dropna(subset=["difference_power_ps_and_calculated_power_ps"])

# Comparar columnas relacionadas
print("¿'registration_date' y 'year' son idénticas?", {df["registration_date"].equals(df["year"])})

if (df["difference_power_ps_and_calculated_power_ps"] == 0).all():
    print("\n'power_kw' y 'power_ps' son idénticas. " \
    "\nValores únicos de la diferencia entre 'power_ps' y 'calulated_power_ps':",
    df["difference_power_ps_and_calculated_power_ps"].unique())
else:
    print("'power_kw' y 'power_ps' no son idénticas.")

# Inspeccionar columnas de modelo y marca
print("\nModelos únicos:", df["model"].unique())
print("\nMarcas únicas:", df["brand"].unique())

¿'registration_date' y 'year' son idénticas? {True}

'power_kw' y 'power_ps' son idénticas. 
Valores únicos de la diferencia entre 'power_ps' y 'calulated_power_ps': [0.]

Modelos únicos: ['Alfa Romeo GTV' 'Alfa Romeo 164' 'Alfa Romeo Spider' ... 'Volvo XC40'
 'Volvo C40' 'Volvo']

Marcas únicas: ['alfa-romeo' 'aston-martin' 'audi' 'bentley' 'bmw' 'cadillac' 'chevrolet'
 'chrysler' 'citroen' 'dacia' 'daewoo' 'daihatsu' 'dodge' 'ferrari' 'fiat'
 'ford' 'honda' 'hyundai' 'infiniti' 'isuzu' 'jaguar' 'jeep' 'kia' 'lada'
 'lamborghini' 'lancia' 'land-rover' 'maserati' 'mazda' 'mercedes-benz'
 'mini' 'mitsubishi' 'nissan' 'opel' 'peugeot' 'porsche' 'proton'
 'renault' 'rover' 'saab' 'seat' 'skoda' 'smart' 'ssangyong' 'toyota'
 'volkswagen' 'volvo']


In [ ]:
# Obtención de la edad del vehículo
current_reference_year = df["year"].max() 
df["age"] = (current_reference_year - df["year"]).astype(int)

In [12]:
current_reference_year

np.float64(2023.0)

In [13]:
# Revisar el DataFrame después de las modificaciones
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250752 entries, 0 to 251078
Data columns (total 17 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   brand                                        250752 non-null  object 
 1   model                                        250752 non-null  object 
 2   color                                        250586 non-null  object 
 3   registration_date                            250752 non-null  float64
 4   year                                         250752 non-null  float64
 5   price_in_euro                                250752 non-null  float64
 6   power_kw                                     250752 non-null  float64
 7   power_ps                                     250752 non-null  float64
 8   transmission_type                            250752 non-null  object 
 9   fuel_type                                    250752 non-null  ob

In [14]:
# Valores nulos después de las modificaciones 
df["price_in_euro"] = df["price_in_euro"].where(df["price_in_euro"] >= 0)

# Eliminar duplicados
df = df.drop_duplicates()

In [15]:
# Eliminar filas con valores nulos en columnas clave
cols_to_drop_nulls = [
    "color", "year", "price_in_euro", "power_kw", "power_ps",
    "fuel_consumption_l_100km", "fuel_consumption_g_km", "mileage_in_km"
]
df = df.dropna(subset=cols_to_drop_nulls)

# Eliminar duplicados
df = df.drop_duplicates()

# Eliminar columnas redundantes o irrelevantes
df = df.drop(columns=["registration_date", "year", "offer_description", "calculated_power_ps", "difference_power_ps_and_calculated_power_ps", "power_ps"])

# Validar nuevamente los valores nulos
print("Valores nulos después de eliminar filas:")
print(df.isnull().sum())

Valores nulos después de eliminar filas:
brand                       0
model                       0
color                       0
price_in_euro               0
power_kw                    0
transmission_type           0
fuel_type                   0
fuel_consumption_l_100km    0
fuel_consumption_g_km       0
mileage_in_km               0
age                         0
dtype: int64


In [16]:
# Guardar el conjunto de datos limpio
path_file_cleaned_data = "../data/processed/cleaned_data.csv"

df.to_csv(path_file_cleaned_data, index=False)

df

,brand,model,color,price_in_euro,power_kw,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,age
0,alfa-romeo,Alfa Romeo GTV,red,1300.0,148.0,Manual,Petrol,10.9,260.0,160500.0,28
3,alfa-romeo,Alfa Romeo Spider,black,4900.0,110.0,Manual,Petrol,9.5,225.0,189500.0,28
5,alfa-romeo,Alfa Romeo Spider,red,7900.0,110.0,Manual,Petrol,9.5,225.0,47307.0,27
6,alfa-romeo,Alfa Romeo 145,red,3500.0,110.0,Manual,Petrol,8.8,210.0,230000.0,27
7,alfa-romeo,Alfa Romeo 164,black,5500.0,132.0,Manual,Petrol,13.4,320.0,168000.0,27
...,...,...,...,...,...,...,...,...,...,...,...
251071,volvo,Volvo XC60,silver,81350.0,228.0,Automatic,Hybrid,7.4,167.0,60.0,0
251072,volvo,Volvo XC60,silver,55400.0,145.0,Automatic,Diesel,5.6,142.0,5000.0,0
251073,volvo,Volvo XC60,silver,54500.0,145.0,Automatic,Diesel,5.6,142.0,5900.0,0
251075,volvo,Volvo XC90,white,89690.0,173.0,Automatic,Diesel,7.6,202.0,4900.0,0
